# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7f607eeca0>
├── 'a' --> tensor([[-0.5353,  1.4278,  1.2774],
│                   [ 1.5311, -0.0027,  0.7318]])
└── 'x' --> <FastTreeValue 0x7f7f607eec40>
    └── 'c' --> tensor([[-0.1010,  0.4470, -1.4695,  0.5388],
                        [ 0.5886, -0.8190,  0.3321, -0.2638],
                        [-0.0428, -0.3900,  0.7487,  2.0655]])

In [4]:
t.a

tensor([[-0.5353,  1.4278,  1.2774],
        [ 1.5311, -0.0027,  0.7318]])

In [5]:
%timeit t.a

60.3 ns ± 0.0553 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7f607eeca0>
├── 'a' --> tensor([[-1.6235,  0.3010,  0.4985],
│                   [ 0.7015,  1.5882,  1.0499]])
└── 'x' --> <FastTreeValue 0x7f7f607eec40>
    └── 'c' --> tensor([[-0.1010,  0.4470, -1.4695,  0.5388],
                        [ 0.5886, -0.8190,  0.3321, -0.2638],
                        [-0.0428, -0.3900,  0.7487,  2.0655]])

In [7]:
%timeit t.a = new_value

66.9 ns ± 0.024 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5353,  1.4278,  1.2774],
               [ 1.5311, -0.0027,  0.7318]]),
    x: Batch(
           c: tensor([[-0.1010,  0.4470, -1.4695,  0.5388],
                      [ 0.5886, -0.8190,  0.3321, -0.2638],
                      [-0.0428, -0.3900,  0.7487,  2.0655]]),
       ),
)

In [10]:
b.a

tensor([[-0.5353,  1.4278,  1.2774],
        [ 1.5311, -0.0027,  0.7318]])

In [11]:
%timeit b.a

56.1 ns ± 0.0273 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0681, -0.3653, -0.4329],
               [ 0.8683, -1.3407, -0.4207]]),
    x: Batch(
           c: tensor([[-0.1010,  0.4470, -1.4695,  0.5388],
                      [ 0.5886, -0.8190,  0.3321, -0.2638],
                      [-0.0428, -0.3900,  0.7487,  2.0655]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.122 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

833 ns ± 3.14 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 49.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 758 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 367 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7f607e7a60>
├── 'a' --> tensor([[[-0.5353,  1.4278,  1.2774],
│                    [ 1.5311, -0.0027,  0.7318]],
│           
│                   [[-0.5353,  1.4278,  1.2774],
│                    [ 1.5311, -0.0027,  0.7318]],
│           
│                   [[-0.5353,  1.4278,  1.2774],
│                    [ 1.5311, -0.0027,  0.7318]],
│           
│                   [[-0.5353,  1.4278,  1.2774],
│                    [ 1.5311, -0.0027,  0.7318]],
│           
│                   [[-0.5353,  1.4278,  1.2774],
│                    [ 1.5311, -0.0027,  0.7318]],
│           
│                   [[-0.5353,  1.4278,  1.2774],
│                    [ 1.5311, -0.0027,  0.7318]],
│           
│                   [[-0.5353,  1.4278,  1.2774],
│                    [ 1.5311, -0.0027,  0.7318]],
│           
│                   [[-0.5353,  1.4278,  1.2774],
│                    [ 1.5311, -0.0027,  0.7318]]])
└── 'x' --> <FastTreeValue 0x7f7eb4a8ebe0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 81.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7ebb0c4910>
├── 'a' --> tensor([[-0.5353,  1.4278,  1.2774],
│                   [ 1.5311, -0.0027,  0.7318],
│                   [-0.5353,  1.4278,  1.2774],
│                   [ 1.5311, -0.0027,  0.7318],
│                   [-0.5353,  1.4278,  1.2774],
│                   [ 1.5311, -0.0027,  0.7318],
│                   [-0.5353,  1.4278,  1.2774],
│                   [ 1.5311, -0.0027,  0.7318],
│                   [-0.5353,  1.4278,  1.2774],
│                   [ 1.5311, -0.0027,  0.7318],
│                   [-0.5353,  1.4278,  1.2774],
│                   [ 1.5311, -0.0027,  0.7318],
│                   [-0.5353,  1.4278,  1.2774],
│                   [ 1.5311, -0.0027,  0.7318],
│                   [-0.5353,  1.4278,  1.2774],
│                   [ 1.5311, -0.0027,  0.7318]])
└── 'x' --> <FastTreeValue 0x7f7ebb0c4760>
    └── 'c' --> tensor([[-0.1010,  0.4470, -1.4695,  0.5388],
                        [ 0.5886, -0.8190,  0.3321, -0.2638],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 40.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.7 µs ± 894 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.5353,  1.4278,  1.2774],
                [ 1.5311, -0.0027,  0.7318]],
       
               [[-0.5353,  1.4278,  1.2774],
                [ 1.5311, -0.0027,  0.7318]],
       
               [[-0.5353,  1.4278,  1.2774],
                [ 1.5311, -0.0027,  0.7318]],
       
               [[-0.5353,  1.4278,  1.2774],
                [ 1.5311, -0.0027,  0.7318]],
       
               [[-0.5353,  1.4278,  1.2774],
                [ 1.5311, -0.0027,  0.7318]],
       
               [[-0.5353,  1.4278,  1.2774],
                [ 1.5311, -0.0027,  0.7318]],
       
               [[-0.5353,  1.4278,  1.2774],
                [ 1.5311, -0.0027,  0.7318]],
       
               [[-0.5353,  1.4278,  1.2774],
                [ 1.5311, -0.0027,  0.7318]]]),
    x: Batch(
           c: tensor([[[-0.1010,  0.4470, -1.4695,  0.5388],
                       [ 0.5886, -0.8190,  0.3321, -0.2638],
                       [-0.0428, -0.3900,  0.7487,  2.0655]],
         

In [26]:
%timeit Batch.stack(batches)

78.5 µs ± 372 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.5353,  1.4278,  1.2774],
               [ 1.5311, -0.0027,  0.7318],
               [-0.5353,  1.4278,  1.2774],
               [ 1.5311, -0.0027,  0.7318],
               [-0.5353,  1.4278,  1.2774],
               [ 1.5311, -0.0027,  0.7318],
               [-0.5353,  1.4278,  1.2774],
               [ 1.5311, -0.0027,  0.7318],
               [-0.5353,  1.4278,  1.2774],
               [ 1.5311, -0.0027,  0.7318],
               [-0.5353,  1.4278,  1.2774],
               [ 1.5311, -0.0027,  0.7318],
               [-0.5353,  1.4278,  1.2774],
               [ 1.5311, -0.0027,  0.7318],
               [-0.5353,  1.4278,  1.2774],
               [ 1.5311, -0.0027,  0.7318]]),
    x: Batch(
           c: tensor([[-0.1010,  0.4470, -1.4695,  0.5388],
                      [ 0.5886, -0.8190,  0.3321, -0.2638],
                      [-0.0428, -0.3900,  0.7487,  2.0655],
                      [-0.1010,  0.4470, -1.4695,  0.5388],
                      [ 0.5886, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 287 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

304 µs ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
